# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing Turi Create functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up Turi Create

Make sure you have the latest version of Turi Create.

In [1]:
from __future__ import division
import math
import string
import pandas as pd
import gdown
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
#!gdown -q 189CTuY0QHPoGrwE0fRLnAqWaYfa5NT7j -O ./amazon_baby.csv

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = pd.read_csv("amazon_baby.csv")

Now, let us see a preview of what the dataset looks like.

In [3]:
products

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5


Drop rows which have missing review

In [64]:
#products = products[products['review'].notna()]

In [65]:
#products.shape

(182702, 3)

## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products.iloc[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
import string 
def remove_punctuation(text):
    try: # python 2.x
        text = text.translate(None, string.punctuation) 
    except: # python 3.x
        translator = str(text).maketrans('', '', string.punctuation)
        text = str(text).translate(translator)
    return text

review_without_punctuation = products['review'].apply(remove_punctuation)
#products['word_count'] = turicreate.text_analytics.count_words(review_without_punctuation)
products['review_clean'] = review_without_punctuation

In [6]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
vectorizer.fit(review_without_punctuation)
tokenizer = vectorizer.build_tokenizer()

In [7]:
def create_word_count(text, tokenizer):
    tokens = tokenizer(text)
    word_count = {}
  
    for token in tokens:
        if token in word_count:
            word_count[token] += 1
        else:
            word_count[token] = 1
      
    return word_count

word_count = []
for review in review_without_punctuation:
    word_count.append(create_word_count(review, tokenizer))
products['word_count'] = word_count

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [8]:
products.iloc[269]['word_count']

{'A': 1, 'favorite': 1, 'in': 1, 'our': 1, 'house': 1}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [106]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

In [9]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [109]:
products['sentiment'] = products['rating'].apply(lambda rating : 1 if rating > 3 else -1)
products

,name,review,rating,review_clean,word_count,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{'it': 3, 'came': 1, 'early': 1, 'and': 3, 'wa...",1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"{'Very': 1, 'soft': 1, 'and': 2, 'comfortable'...",1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"{'This': 1, 'is': 4, 'a': 2, 'product': 2, 'we...",1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"{'All': 1, 'of': 1, 'my': 1, 'kids': 2, 'have'...",1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"{'When': 1, 'the': 5, 'Binky': 3, 'Fairy': 3, ...",1
...,...,...,...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5,Such a great idea very handy to have and look ...,"{'Such': 1, 'a': 1, 'great': 2, 'idea': 1, 've...",1
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5,This product rocks It is a great blend of fun...,"{'This': 1, 'product': 2, 'rocks': 1, 'It': 1,...",1
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5,This item looks great and cool for my kidsI kn...,"{'This': 1, 'item': 1, 'looks': 1, 'great': 2,...",1
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5,I am extremely happy with this product I have ...,"{'I': 9, 'am': 2, 'extremely': 1, 'happy': 1, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [75]:
#train_data, test_data = train_test_split(products, test_size = 0.2, stratify = products['sentiment'])

In [110]:
# using the train idx and test idx
import json

f = open('module-2-assignment-test-idx.json')
test_idx = json.load(f)

f = open('module-2-assignment-train-idx.json')
train_idx = json.load(f)

train_data = products.iloc[train_idx]
test_data = products.iloc[test_idx]

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')

train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [ ]:
sentiment_model = turicreate.logistic_classifier.create(train_data,
                                                        target = 'sentiment',
                                                        features=['word_count'],
                                                        validation_set=None)

Because I could not use turicreate so I will use sklearn instead

In [123]:
sentiment_model = LogisticRegression().fit(X = train_matrix,
                                           y = train_data['sentiment'])

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [124]:
weights = sentiment_model.coef_[0]
len(weights)

121713

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [125]:
num_positive_weights = (weights >= 0).sum()
num_negative_weights = len(weights) - num_positive_weights

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 90262 
Number of negative weights: 31451 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [126]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

59    5
71    2
91    1
Name: rating, dtype: int64


,name,review,rating,review_clean,word_count,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,"{'Absolutely': 1, 'love': 1, 'it': 2, 'and': 2...",1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,"{'Would': 1, 'not': 1, 'purchase': 1, 'again':...",-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,"{'Was': 1, 'so': 1, 'excited': 1, 'to': 3, 'ge...",-1


In [127]:
test_data[10:13]

,name,review,rating,review_clean,word_count,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,"{'Absolutely': 1, 'love': 1, 'it': 2, 'and': 2...",1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,"{'Would': 1, 'not': 1, 'purchase': 1, 'again':...",-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,"{'Was': 1, 'so': 1, 'excited': 1, 'to': 3, 'ge...",-1


In [128]:
products.iloc[[10,11,12]]

,name,review,rating,review_clean,word_count,sentiment
11,"Baby Tracker&reg; - Daily Childcare Journal, S...",This book is perfect! I'm a first time new mo...,5,This book is perfect Im a first time new mom ...,"{'This': 1, 'book': 2, 'is': 1, 'perfect': 1, ...",1
12,"Baby Tracker&reg; - Daily Childcare Journal, S...",I originally just gave the nanny a pad of pape...,4,I originally just gave the nanny a pad of pape...,"{'I': 7, 'originally': 1, 'just': 1, 'gave': 1...",1
14,Nature's Lullabies First Year Sticker Calendar,"Space for monthly photos, info and a lot of us...",5,Space for monthly photos info and a lot of use...,"{'Space': 1, 'for': 1, 'monthly': 1, 'photos':...",1


In [129]:
products.iloc[82
             ]

name            New Style Trailing Cherry Blossom Tree Decal R...
review          Was so excited to get this product for my baby...
rating                                                          1
review_clean    Was so excited to get this product for my baby...
word_count      {'Was': 1, 'so': 1, 'excited': 1, 'to': 3, 'ge...
sentiment                                                      -1
Name: 91, dtype: object

In [102]:
test_idx

[8,
 9,
 14,
 18,
 24,
 31,
 32,
 36,
 38,
 50,
 53,
 64,
 82,
 102,
 105,
 106,
 109,
 112,
 117,
 123,
 126,
 132,
 134,
 142,
 143,
 145,
 149,
 156,
 162,
 165,
 167,
 170,
 175,
 176,
 184,
 195,
 196,
 197,
 198,
 199,
 205,
 217,
 221,
 223,
 226,
 229,
 233,
 234,
 235,
 239,
 256,
 258,
 261,
 263,
 264,
 284,
 285,
 286,
 293,
 326,
 328,
 335,
 338,
 350,
 352,
 356,
 362,
 366,
 370,
 382,
 390,
 398,
 399,
 401,
 403,
 410,
 411,
 412,
 415,
 418,
 419,
 425,
 430,
 431,
 433,
 434,
 437,
 438,
 451,
 453,
 456,
 464,
 466,
 469,
 492,
 504,
 505,
 509,
 511,
 519,
 521,
 526,
 528,
 534,
 542,
 543,
 546,
 551,
 555,
 561,
 562,
 563,
 567,
 574,
 577,
 580,
 581,
 585,
 586,
 591,
 596,
 600,
 612,
 614,
 622,
 623,
 626,
 630,
 635,
 637,
 647,
 651,
 656,
 657,
 661,
 662,
 668,
 671,
 682,
 690,
 691,
 696,
 714,
 715,
 716,
 718,
 722,
 733,
 737,
 740,
 741,
 742,
 765,
 779,
 781,
 784,
 788,
 790,
 793,
 802,
 805,
 817,
 825,
 827,
 845,
 848,
 851,
 853,
 855,
 

Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [42]:
sample_test_data.iloc[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [43]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using Turi Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [130]:
scores = sentiment_model.predict_proba(vectorizer.transform(sample_test_data['review_clean']).toarray())
print(scores)

[[5.62289151e-03 9.94377108e-01]
 [9.60597475e-01 3.94025251e-02]
 [9.99984400e-01 1.56000595e-05]]


In [131]:
sample_test_data

,name,review,rating,review_clean,word_count,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,"{'Absolutely': 1, 'love': 1, 'it': 2, 'and': 2...",1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,"{'Would': 1, 'not': 1, 'purchase': 1, 'again':...",-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,"{'Was': 1, 'so': 1, 'excited': 1, 'to': 3, 'ge...",-1


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [132]:
vectorizer.transform(sample_test_data['review_clean']).toarray().shape

(3, 121713)

In [133]:
weights.shape

(121713,)

In [134]:
def predict(weight, x):
    score = np.dot(x, weight)
    return list(map(lambda x: 1 if x > 0 else -1, score))

predict(weights, vectorizer.transform(sample_test_data['review_clean']).toarray())

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from Turi Create.

In [135]:
print("Class predictions according to SKlearn:")
print(sentiment_model.predict(vectorizer.transform(sample_test_data['review_clean']).toarray()))

Class predictions according to SKlearn:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from Turi Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [136]:
def predict_prob(w, x):
    score = np.dot(x, w)
    prob = 1/(1+np.exp(-score))
    return prob

In [137]:
predict_prob(weights, vectorizer.transform(sample_test_data['review_clean']).toarray())

array([9.80163749e-01, 1.13313331e-02, 4.35891860e-06])

**Checkpoint**: Make sure your probability predictions match the ones obtained from Turi Create.

In [138]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict_proba(vectorizer.transform(sample_test_data['review_clean']).toarray()))

Class predictions according to Turi Create:
[[5.62289151e-03 9.94377108e-01]
 [9.60597475e-01 3.94025251e-02]
 [9.99984400e-01 1.56000595e-05]]


**Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use Turi Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [140]:
top20 = sentiment_model.predict_proba(test_matrix)[:,1].argsort()[-40:][::-1]
set(test_data.iloc[top20]['name'])

{'BABYBJORN Potty Chair - Red',
 'Baby Einstein Around The World Discovery Center',
 'Baby Jogger City Mini GT Double Stroller, Shadow/Orange',
 'Baby Jogger City Mini GT Single Stroller, Shadow/Orange',
 'Britax 2012 B-Agile Stroller, Red',
 'Britax Boulevard 70-G3 Convertible Car Seat Seat, Onyx',
 'Britax Decathlon Convertible Car Seat, Tiffany',
 'Britax Frontier Booster Car Seat',
 'Buttons Cloth Diaper Cover - One Size - 8 Color Options',
 'Delta Universal 6 Drawer Dresser, Black Cherry',
 'Diono RadianRXT Convertible Car Seat, Plum',
 'EZ Squeezees Refillable Food Pouches,sold in pack of 3. 3 pouches each',
 'Emily Green 6&quot; Bowl, Sunshine Safari',
 'Evenflo 6 Pack Classic Glass Bottle, 4-Ounce',
 'Evenflo X Sport Plus Convenience Stroller - Christina',
 "Fisher-Price Cradle 'N Swing,  My Little Snugabunny",
 'Fisher-Price Zen Collection Cradle Swing',
 'Freemie Hands-Free Concealable Breast Pump Collection System',
 'Graco FastAction Fold Jogger Click Connect Stroller, Grap

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [149]:
top20_ = sentiment_model.predict_proba(test_matrix)[:,1].argsort()[:15]
test_data.iloc[top20_]['name']

94560     The First Years True Choice P400 Premium Digit...
16042           Fisher-Price Ocean Wonders Aquarium Bouncer
120209    Levana Safe N'See Digital Video Baby Monitor w...
155287    VTech Communications Safe &amp; Sounds Full Co...
53207                   Safety 1st High-Def Digital Monitor
167249    Samsung SEW-3037W Wireless Pan Tilt Video Baby...
81332                 Cloth Diaper Sprayer--styles may vary
48694     Adiri BPA Free Natural Nurser Ultimate Bottle ...
176046         Baby Trend Inertia Infant Car Seat - Horizon
59546                Ellaroo Mei Tai Baby Carrier - Hershey
95420          One Step Ahead Hide-Away Extra Long Bed Rail
77072        Safety 1st Exchangeable Tip 3 in 1 Thermometer
96572      Baby Jogger Summit XC Double Stroller, Red/Black
94389                  Snuza Portable Baby Movement Monitor
76000            Peg-Perego Tatamia High Chair, White Latte
Name: name, dtype: object

In [151]:
sentiment_model.predict_proba(test_matrix)[:,1][top20_]

array([1.49867426e-15, 2.21259188e-15, 2.13904525e-13, 1.19073927e-12,
       6.27557995e-11, 1.14304815e-10, 1.31534025e-10, 1.57376501e-10,
       1.74132430e-10, 1.83164621e-10, 2.10702361e-10, 2.15079418e-10,
       2.48821777e-10, 3.48448997e-10, 4.01128228e-10])

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [146]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    pred = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    n_correct = (pred == true_labels).sum()
    
    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = n_correct/len(true_labels)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [147]:
print(get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment']))
print(get_classification_accuracy(sentiment_model, train_matrix, train_data['sentiment']))

0.9319654427645788
0.9478698207111591


**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [31]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [32]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [34]:
def dict_trim_by_keys(mydict, List):
    tmp = {}
    
    for key in mydict.keys():
        if key in List:
            tmp[key] = mydict[key]
    
    return tmp
    

train_data['word_count_subset'] = train_data['word_count'].apply(lambda x: dict_trim_by_keys(x, significant_words))
test_data['word_count_subset'] = test_data['word_count'].apply(lambda x: dict_trim_by_keys(x, significant_words))

Let's see what the first example of the dataset looks like:

In [36]:
train_data.iloc[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [37]:
print(train_data.iloc[0]['word_count'])

{'it': 3, 'came': 1, 'early': 1, 'and': 3, 'was': 1, 'not': 2, 'disappointed': 1, 'i': 1, 'love': 1, 'planet': 1, 'wise': 1, 'bags': 1, 'now': 1, 'my': 2, 'wipe': 1, 'holder': 1, 'keps': 1, 'osocozy': 1, 'wipes': 1, 'moist': 1, 'does': 1, 'leak': 1, 'highly': 1, 'recommend': 1}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [38]:
print(train_data.iloc[0]['word_count_subset'])

{'disappointed': 1, 'love': 1}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [ ]:
simple_model = turicreate.logistic_classifier.create(train_data,
                                                     target = 'sentiment',
                                                     features=['word_count_subset'],
                                                     validation_set=None)

In [46]:
def simplify(text, significant_words):
    return ' '.join([x for x in text.split() if x in significant_words])
    
simple_train_data = train_data['review_clean'].apply(lambda x: simplify(x,significant_words))
simple_test_data = test_data['review_clean'].apply(lambda x: simplify(x,significant_words))

simple_vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')

simple_train_matrix = vectorizer.fit_transform(simple_train_data)
simple_test_matrix = vectorizer.transform(simple_test_data)

In [47]:
simple_model = LogisticRegression().fit(X = simple_train_matrix,
                                           y = train_data['sentiment'])

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [48]:
#get_classification_accuracy(simple_model, test_data, test_data['sentiment'])
get_classification_accuracy(simple_model, simple_test_matrix, test_data['sentiment'])

0.8673506119510439

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [60]:
simple_model.coef_[0]

array([ 0.18634918, -1.64844682,  0.07445436, -2.31404997,  1.13217528,
       -0.51084026,  0.85266174, -0.21736871,  0.54211338,  1.29752527,
        1.67419808, -0.933143  ,  0.09912872,  1.44165452, -0.29823398,
       -2.08905855, -1.91959733,  0.55693256, -0.63570201, -0.38790139])

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [58]:
vectorizer.fit(simple_train_data)
vectorizer.get_feature_names_out()

array(['able', 'broke', 'car', 'disappointed', 'easy', 'even', 'great',
       'less', 'little', 'love', 'loves', 'money', 'old', 'perfect',
       'product', 'return', 'waste', 'well', 'work', 'would'],
      dtype=object)

In [75]:
#simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)
coef_idx = simple_model.coef_[0].argsort()[-20:][::-1]
feature_names = vectorizer.get_feature_names_out()[coef_idx]
feature_coef = [{key, value} for key, value in zip(feature_names, simple_model.coef_[0][coef_idx])]

feature_coef

[{1.6741980788170672, 'loves'},
 {1.4416545217609096, 'perfect'},
 {1.2975252689896872, 'love'},
 {1.1321752792601463, 'easy'},
 {0.8526617429803622, 'great'},
 {0.5569325573662659, 'well'},
 {0.5421133808158118, 'little'},
 {0.18634918424700583, 'able'},
 {0.09912871688257827, 'old'},
 {0.07445435660225573, 'car'},
 {-0.21736870809605957, 'less'},
 {-0.29823397973218413, 'product'},
 {-0.38790138825475795, 'would'},
 {-0.5108402550690643, 'even'},
 {-0.6357020105459842, 'work'},
 {-0.9331429961616524, 'money'},
 {-1.6484468158007455, 'broke'},
 {-1.919597326961822, 'waste'},
 {-2.0890585546577625, 'return'},
 {-2.314049973706252, 'disappointed'}]

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [76]:
(simple_model.coef_[0] > 0).sum()

10

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [84]:
positive_words =  feature_names[:10]

In [85]:
positive_words_idx = list(map(lambda x: list(vectorizer.get_feature_names_out()).index(x), positive_words))
positive_words_idx_coef_ = sentiment_model.coef_[0][positive_words_idx]

In [86]:
positive_words_idx_coef_

array([ 1.77226011,  2.02081328,  1.60505608,  1.38917734,  1.27450867,
        0.48203093,  0.54505152,  0.32870157, -0.02069284,  0.11950963])

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [87]:
get_classification_accuracy(sentiment_model, train_matrix, train_data['sentiment'])

0.9478698207111591

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [89]:
get_classification_accuracy(simple_model, simple_train_matrix, train_data['sentiment'])

0.8644540384961324

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [91]:
get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

0.9319654427645788

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [90]:
get_classification_accuracy(simple_model, simple_test_matrix, test_data['sentiment'])

0.8673506119510439

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [92]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [93]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)
num_positive/len(test_data)

28095
5241


0.8427825773938085

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?